# Continous Growth - Proportional Bid/Ask-Spread

The model is in accordance with the model used in CHordia(2000) with the exemptance that I left the squared returns out. My model is in log - log form whereas their model is in precentage change. 

The model is as follows (on continous growth form): 
                                                    DLjt = @ + DLMt + DLMt-1 + DLMt+1 + e
With: DLjt = Change in liquidity of security j at time t
      DLMt = Change in liquidity of the market j at time t
      DLMt-1 = 1 period - lagged variable of DLMt
      DLMt+1 = 1 period lead variable of DLMt
      
I also have an additional model which is as follows: DLjt = @ + DLMt + DLMt-1 + DLMt+1 + DLjt^2 + e
###### However, this model displays very strong heteroscedasticity due to DLjt^2.
I also added 1 and 2 period lagged variables of dependent variable in other models which fixed autocorrelation but I do not know if it is necessary as discussed. 

Let me know what you think at your convenience.
                                                        
      
      

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.stats.stattools import durbin_watson

In [2]:
pd.set_option('use_inf_as_na', True)

In [3]:
df=pd.read_excel("BBREIT-1.07.xlsx",sheet_name="PROPORTIONAL_BID_ASK_SPREAD", index_col="Dates")
df.dropna(axis=1,inplace=True)

In [4]:
for i in df:
    df[i]= np.log(df[i])-np.log(df[i]).shift(1)

C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
df.drop(index=df.index[0], axis=0, inplace=True)

In [6]:
df.isnull().values.sum()

55

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().values.sum()

0

In [9]:
q =df.quantile(0.9999)
b =df.quantile(0.0001)

In [10]:
df=df[(df < q) & (df > b)]

In [11]:
df["Sum"]=df.sum(axis=1)

In [19]:
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    
    df["M"]=((df["Sum"]-df[i])/157) # Cross Sectional Mean Variable
    df["Mlag"]=df["M"].shift(1)
    df["Mlead"]=df["M"].shift(-1)
    df.replace([np.inf], np.nan, inplace=True)
    df=df.fillna(0)
        
        
      
    y = df[i]
    x = df[["M","Mlag","Mlead"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","Mlag","Mlead"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     7.145
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           0.000104
Time:                        14:03:21   Log-Likelihood:                -953.84
No. Observations:                 516   AIC:                             1916.
Df Residuals:                     512   BIC:                             1933.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0098      0.068     -0.143      0.8

(2.673944372967221, 0.44467335051153123, 0.8890122914742021, 0.4466017408545565)
2.8975652430409187
                            OLS Regression Results                            
Dep. Variable:          EXR US Equity   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.206
Date:                Sat, 22 Oct 2022   Prob (F-statistic):            0.00592
Time:                        14:03:21   Log-Likelihood:                -870.92
No. Observations:                 516   AIC:                             1750.
Df Residuals:                     512   BIC:                             1767.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(4.965148486440684, 0.17436700401498179, 1.658175248079719, 0.17509659109414252)
2.853959366182253
                            OLS Regression Results                            
Dep. Variable:          CPT US Equity   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.832
Date:                Sat, 22 Oct 2022   Prob (F-statistic):             0.0378
Time:                        14:03:21   Log-Likelihood:                -857.29
No. Observations:                 516   AIC:                             1723.
Df Residuals:                     512   BIC:                             1740.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(10.652751336663524, 0.013759705142596283, 3.5976639157848456, 0.013518352575007164)
3.046767691020788
                            OLS Regression Results                            
Dep. Variable:           HR US Equity   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     5.085
Date:                Sat, 22 Oct 2022   Prob (F-statistic):            0.00178
Time:                        14:03:21   Log-Likelihood:                -399.38
No. Observations:                 516   AIC:                             806.8
Df Residuals:                     512   BIC:                             823.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

(5.523956709492157, 0.13721274170478354, 1.8468159099945782, 0.13768135146001195)
2.9146574526245477
                            OLS Regression Results                            
Dep. Variable:          SRC US Equity   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     20.58
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.32e-12
Time:                        14:03:21   Log-Likelihood:                -658.66
No. Observations:                 516   AIC:                             1325.
Df Residuals:                     512   BIC:                             1342.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

...

                            OLS Regression Results                            
Dep. Variable:          UBA US Equity   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     3.819
Date:                Sat, 22 Oct 2022   Prob (F-statistic):             0.0100
Time:                        14:03:23   Log-Likelihood:                -603.58
No. Observations:                 516   AIC:                             1215.
Df Residuals:                     512   BIC:                             1232.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0185      0.035     -0.535      0.5

(7.854053050548991, 0.04912628085903584, 2.6378741422761034, 0.04897918036496568)
2.813646711242687
                            OLS Regression Results                            
Dep. Variable:         PINE US Equity   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     4.553
Date:                Sat, 22 Oct 2022   Prob (F-statistic):            0.00368
Time:                        14:03:23   Log-Likelihood:                -769.00
No. Observations:                 516   AIC:                             1546.
Df Residuals:                     512   BIC:                             1563.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(10.819470882878699, 0.012743212122766402, 3.6551745846282744, 0.01250567503688667)
2.814130942301134
                            OLS Regression Results                            
Dep. Variable:          PEI US Equity   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     3.602
Date:                Sat, 22 Oct 2022   Prob (F-statistic):             0.0134
Time:                        14:03:24   Log-Likelihood:                -843.31
No. Observations:                 516   AIC:                             1695.
Df Residuals:                     512   BIC:                             1712.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

C:\Users\ravn_\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


KeyError: 'M'

## Summary Results

This looks like the most promising model so far. I removed the squared returns which in place increased the significance level of which we can reject homoscedasticity from 0% to 36%. I added the squared returns in the model below which in turn exploded the heteroscedasticity. Unless there are anything obvious that I am missing, I believe this is the best model and the one I will base the written part around.

In [20]:
Coef=pd.DataFrame(Coefficients)
round(Coef["M"].mean(),4)

0.8179

In [21]:
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.0345
dtype: float64

In [22]:
DBW=pd.DataFrame(DBWatson)
round(DBW.mean(),4)

0    2.876
dtype: float64

In [27]:
PV=pd.DataFrame(Pvalues)
round(PV["M"].mean(),4)

0.092

In [24]:
BP=pd.DataFrame(Bpagan)
round(BP[1].mean(),4)

0.3595

## Regression with squared returns

In [43]:
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    df["M"]=((df["Sum"]-df[i])/157) # Cross Sectional Mean Variable
    df["Mlag"]=df["M"].shift(1)
    df["Mlead"]=df["M"].shift(-1)
    df["VOL"]=df[i]**2
    df.replace([np.inf], np.nan, inplace=True)
    df=df.fillna(0)
        
        
      
    y = df[i]
    x = df[["M","Mlag","Mlead","VOL"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","Mlag","Mlead","VOL"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     5.353
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           0.000314
Time:                        14:31:37   Log-Likelihood:                -953.83
No. Observations:                 516   AIC:                             1918.
Df Residuals:                     511   BIC:                             1939.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0173      0.089     -0.195      0.8

(463.03733740597437, 6.5944344011428905e-99, 1116.8815342053051, 5.682442677913753e-251)
2.82809942324555
                            OLS Regression Results                            
Dep. Variable:          ARE US Equity   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     15.89
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           2.89e-12
Time:                        14:31:37   Log-Likelihood:                -923.96
No. Observations:                 516   AIC:                             1858.
Df Residuals:                     511   BIC:                             1879.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

(504.8932433958337, 5.8574923243682415e-108, 5807.28597398294, 0.0)
2.943026320766695
                            OLS Regression Results                            
Dep. Variable:         PEAK US Equity   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     8.892
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           6.02e-07
Time:                        14:31:37   Log-Likelihood:                -158.24
No. Observations:                 516   AIC:                             326.5
Df Residuals:                     511   BIC:                             347.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

(377.91509787938867, 1.6422366446248686e-80, 349.63021309833385, 9.995500582825803e-145)
2.4734193218235703
                            OLS Regression Results                            
Dep. Variable:         REXR US Equity   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     10.31
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           4.89e-08
Time:                        14:31:37   Log-Likelihood:                -678.90
No. Observations:                 516   AIC:                             1368.
Df Residuals:                     511   BIC:                             1389.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

(421.40833079245925, 6.577684785540641e-90, 569.1295513627008, 1.1747220543129619e-186)
2.7640135112290465
                            OLS Regression Results                            
Dep. Variable:          OHI US Equity   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     18.66
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           2.52e-14
Time:                        14:31:38   Log-Likelihood:                -389.40
No. Observations:                 516   AIC:                             788.8
Df Residuals:                     511   BIC:                             810.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------

...

                            OLS Regression Results                            
Dep. Variable:         NXDT US Equity   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.3954
Date:                Sat, 22 Oct 2022   Prob (F-statistic):              0.812
Time:                        14:31:39   Log-Likelihood:                -669.15
No. Observations:                 516   AIC:                             1348.
Df Residuals:                     511   BIC:                             1370.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0119      0.048     -0.249      0.8

(476.30958414357667, 8.899529600486301e-102, 1533.0791593229035, 5.710217787251139e-283)
2.742708598256024
                            OLS Regression Results                            
Dep. Variable:           HT US Equity   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     4.244
Date:                Sat, 22 Oct 2022   Prob (F-statistic):            0.00218
Time:                        14:31:40   Log-Likelihood:                -409.44
No. Observations:                 516   AIC:                             828.9
Df Residuals:                     511   BIC:                             850.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------

                            OLS Regression Results                            
Dep. Variable:          VOC US Equity   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     2.156
Date:                Sat, 22 Oct 2022   Prob (F-statistic):             0.0728
Time:                        14:31:40   Log-Likelihood:                -601.26
No. Observations:                 516   AIC:                             1213.
Df Residuals:                     511   BIC:                             1234.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0576      0.041     -1.409      0.1

(491.19177444661744, 5.383283229430278e-105, 2529.392884248411, 0.0)
2.896515862714831
                            OLS Regression Results                            
Dep. Variable:         CORR US Equity   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     3.748
Date:                Sat, 22 Oct 2022   Prob (F-statistic):            0.00512
Time:                        14:31:40   Log-Likelihood:                -612.78
No. Observations:                 516   AIC:                             1236.
Df Residuals:                     511   BIC:                             1257.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------

C:\Users\ravn_\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


## Results Summary

This model produces very similar results to the previous one with the exemtion of Heteroscedasticity going through the roof.

In [44]:
Coef=pd.DataFrame(Coefficients)
round(Coef["M"].mean(),4)

0.8086

In [45]:
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.0417
dtype: float64

In [46]:
DBW=pd.DataFrame(DBWatson)
round(DBW.mean(),4)

0    2.8746
dtype: float64

In [47]:
PV=pd.DataFrame(Pvalues)
round(PV["M"].mean(),4)

0.0949

In [48]:
BP=pd.DataFrame(Bpagan)
round(BP[1].mean(),4)

0.0